<p align="center"><img src="https://raw.githubusercontent.com/rxavier/poniard/main/logo.png" alt="Poniard logo" title="Poniard" width="50%"/></p>

# Poniard

> Streamline scikit-learn model comparison.

## Introduction
> A poniard /ˈpɒnjərd/ or poignard (Fr.) is a long, lightweight thrusting knife ([Wikipedia](https://en.wikipedia.org/wiki/Poignard)).

Poniard is a scikit-learn companion library that streamlines the process of fitting different machine learning models and comparing them. 

It can be used to provide quick answers to questions like these:
* What is the reasonable range of scores for this task?
* Is a simple and explainable linear model enough or should I work with forests and gradient boosters?
* Are the features good enough as is or should I work on feature engineering?
* How much can hyperparemeter tuning improve metrics?
* Do I need to work on a custom preprocessing strategy?

This is not meant to be end to end solution, and you definitely should keep on working on your models after you are done with Poniard.

The core functionality has been tested to work on Python 3.7 through 3.10 on Linux systems, and from
3.8 to 3.10 on macOS.

## Installation

Stable version:

```bash
pip install poniard
```

Dev version with most up to date changes:

```bash
pip install git+https://github.com/rxavier/poniard.git@develop#egg=poniard
```

## Documentation

Check the full [Quarto docs](https://rxavier.github.io/poniard), including guides and API reference.

## Usage/features

### Basics
The API was designed with tabular tasks in mind, but it should also work with time series tasks provided an appropiate cross validation strategy is used (don't shuffle!)

The usual Poniard flow is:
1. Define some estimators.
2. Define some metrics.
3. Define a cross validation strategy.
4. Fit everything.
5. Print the results.

Poniard provides sane defaults for 1, 2 and 3, so in most cases you can just do...


In [ ]:
from poniard import PoniardClassifier
from sklearn.datasets import load_breast_cancer

In [ ]:
X, y = load_breast_cancer(return_X_y=True, as_frame=True)
pnd = PoniardClassifier(random_state=0)
pnd.setup(X, y)
pnd.fit()

Target info
-----------
Type: binary
Shape: (569,)
Unique values: 2

Main metric
-----------
roc_auc

Thresholds
----------
Minimum unique values to consider a feature numeric: 56
Minimum unique values to consider a categorical high cardinality: 20

Inferred feature types
----------------------


,numeric,categorical_high,categorical_low,datetime
0,mean radius,,,
1,mean texture,,,
2,mean perimeter,,,
3,mean area,,,
4,mean smoothness,,,
5,mean compactness,,,
6,mean concavity,,,
7,mean concave points,,,
8,mean symmetry,,,
9,mean fractal dimension,,,


  0%|          | 0/9 [00:00<?, ?it/s]

PoniardClassifier(random_state=0)

... and get a nice table showing the average of each metric in all folds for every model, including fit and score times (thanks, scikit-learn `cross_validate` function!)

In [ ]:
# |output: false

pnd.get_results()

,test_roc_auc,test_accuracy,test_precision,test_recall,test_f1,fit_time,score_time
LogisticRegression,0.995456,0.978916,0.975411,0.991549,0.983351,0.006384,0.004307
SVC,0.994139,0.975408,0.975111,0.985955,0.980477,0.014951,0.003119
HistGradientBoostingClassifier,0.994128,0.970129,0.967263,0.985955,0.976433,0.485832,0.019794
XGBClassifier,0.994123,0.970129,0.967554,0.985915,0.976469,0.049917,0.003516
RandomForestClassifier,0.992264,0.964881,0.964647,0.980282,0.972192,0.073579,0.008674
GaussianNB,0.988730,0.929700,0.940993,0.949413,0.944300,0.001976,0.003919
KNeighborsClassifier,0.980610,0.964881,0.955018,0.991628,0.972746,0.002964,0.021620
DecisionTreeClassifier,0.920983,0.926223,0.941672,0.941080,0.941054,0.005010,0.002273
DummyClassifier,0.500000,0.627418,0.627418,1.000000,0.771052,0.001688,0.002320


In [ ]:
# |echo: false
from IPython.display import display, HTML

res = pnd.get_results()
display(HTML(res.to_html()))

,test_roc_auc,test_accuracy,test_precision,test_recall,test_f1,fit_time,score_time
LogisticRegression,0.995456,0.978916,0.975411,0.991549,0.983351,0.006384,0.004307
SVC,0.994139,0.975408,0.975111,0.985955,0.980477,0.014951,0.003119
HistGradientBoostingClassifier,0.994128,0.970129,0.967263,0.985955,0.976433,0.485832,0.019794
XGBClassifier,0.994123,0.970129,0.967554,0.985915,0.976469,0.049917,0.003516
RandomForestClassifier,0.992264,0.964881,0.964647,0.980282,0.972192,0.073579,0.008674
GaussianNB,0.988730,0.929700,0.940993,0.949413,0.944300,0.001976,0.003919
KNeighborsClassifier,0.980610,0.964881,0.955018,0.991628,0.972746,0.002964,0.021620
DecisionTreeClassifier,0.920983,0.926223,0.941672,0.941080,0.941054,0.005010,0.002273
DummyClassifier,0.500000,0.627418,0.627418,1.000000,0.771052,0.001688,0.002320


Alternatively, you can also get a nice plot of your different metrics by using the `PoniardBaseEstimator.plot.metrics` method.

### Type inference
Poniard uses some basic heuristics to infer the data types.

Float and integer columns are defined as numeric if the number of unique values is greater than indicated by the `categorical_threshold` parameter.

String/object/categorical columns are assumed to be categorical.

Datetime features are processed separately with a custom encoder.

For categorical features, high and low cardinality is defined by the `cardinality_threshold` parameter. Only low cardinality categorical features are one-hot encoded.

### Ensembles
Poniard makes it easy to combine various estimators in stacking or voting ensembles. The base esimators can be selected according to their performance (top-n) or chosen by their names.

Poniard also reports how similar the predictions of the estimators are, so ensembles with different base estimators can be built. A basic correlation table of the cross-validated predictions is built for regression tasks, while [Cramér's V](https://en.wikipedia.org/wiki/Cram%C3%A9r%27s_V) is used for classification.

By default, it computes this similarity of prediction errors instead of the actual predictions; this helps in building ensembles with good scoring estimators and uncorrelated errors, which in principle and hopefully should lead to a "wisdom of crowds" kind of situation.


### Hyperparameter optimization
The `PoniardBaseEstimator.tune_estimator` method can be used to optimize the hyperparameters of a given estimator, either by passing a grid of parameters or using the inbuilt ones available for default estimators. The tuned estimator will be added to the list of estimators and will be scored the next time `PoniardBaseEstimator.fit` is called.

In [ ]:
# |output: false

pnd.tune_estimator("RandomForestClassifier", mode="random")
pnd.fit()  # This will only fit new estimators
pnd.get_results()

  0%|          | 0/1 [00:00<?, ?it/s]

,test_roc_auc,test_accuracy,test_precision,test_recall,test_f1,fit_time,score_time
LogisticRegression,0.995456,0.978916,0.975411,0.991549,0.983351,0.006384,0.004307
SVC,0.994139,0.975408,0.975111,0.985955,0.980477,0.014951,0.003119
HistGradientBoostingClassifier,0.994128,0.970129,0.967263,0.985955,0.976433,0.485832,0.019794
XGBClassifier,0.994123,0.970129,0.967554,0.985915,0.976469,0.049917,0.003516
RandomForestClassifier_tuned,0.992696,0.959618,0.961836,0.974726,0.968054,0.071821,0.008250
RandomForestClassifier,0.992264,0.964881,0.964647,0.980282,0.972192,0.073579,0.008674
GaussianNB,0.988730,0.929700,0.940993,0.949413,0.944300,0.001976,0.003919
KNeighborsClassifier,0.980610,0.964881,0.955018,0.991628,0.972746,0.002964,0.021620
DecisionTreeClassifier,0.920983,0.926223,0.941672,0.941080,0.941054,0.005010,0.002273
DummyClassifier,0.500000,0.627418,0.627418,1.000000,0.771052,0.001688,0.002320


In [ ]:
# |echo: false
from IPython.display import display, HTML

res = pnd.get_results()
display(HTML(res.to_html()))

,test_roc_auc,test_accuracy,test_precision,test_recall,test_f1,fit_time,score_time
LogisticRegression,0.995456,0.978916,0.975411,0.991549,0.983351,0.006384,0.004307
SVC,0.994139,0.975408,0.975111,0.985955,0.980477,0.014951,0.003119
HistGradientBoostingClassifier,0.994128,0.970129,0.967263,0.985955,0.976433,0.485832,0.019794
XGBClassifier,0.994123,0.970129,0.967554,0.985915,0.976469,0.049917,0.003516
RandomForestClassifier_tuned,0.992696,0.959618,0.961836,0.974726,0.968054,0.071821,0.008250
RandomForestClassifier,0.992264,0.964881,0.964647,0.980282,0.972192,0.073579,0.008674
GaussianNB,0.988730,0.929700,0.940993,0.949413,0.944300,0.001976,0.003919
KNeighborsClassifier,0.980610,0.964881,0.955018,0.991628,0.972746,0.002964,0.021620
DecisionTreeClassifier,0.920983,0.926223,0.941672,0.941080,0.941054,0.005010,0.002273
DummyClassifier,0.500000,0.627418,0.627418,1.000000,0.771052,0.001688,0.002320


### Plotting

The `plot` accessor provides several plotting methods based on the attached Poniard estimator instance. These Plotly plots are based on a default template, but can be modified by passing a different `PoniardPlotFactory` to the Poniard `plot_options` argument.

### Plugin system

The `plugins` argument in Poniard estimators takes a plugin or list of plugins that subclass `BasePlugin`. These plugins have access to the Poniard estimator instance and hook onto different sections of the process, for example, on setup start, on fit end, on remove estimator, etc.

This makes it easy for third parties to extend Poniard's functionality.

Two plugins are baked into Poniard.
1. Weights and Biases: logs your data, plots, runs wandb scikit-learn analysis, saves model artifacts, etc.
2. Pandas Profiling: generates an HTML report of the features and target. If the Weights and Biases plugin is present, also logs this report to the wandb run.

The requirements for these plugins are not included in the base Poniard dependencies, so you can safely ignore them if you don't intend to use them.

## Design philosophy

### Not another dependency
We try very hard to avoid cluttering the environment with stuff you won't use outside of this library. Poniard's dependencies are:

1. scikit-learn (duh)
2. pandas
3. XGBoost
4. Plotly
5. tqdm
6. That's it!

Apart from `tqdm` and possibly `Plotly`, all dependencies most likely were going to be installed anyway, so Poniard's added footprint should be small.

### We don't do that here (AutoML)
Poniard tries not to take control away from the user. As such, it is not designed to perform 2 hours of feature engineering and selection, try every model under the sun together with endless ensembles and select the top performing model according to some metric.

Instead, it strives to abstract away some of the boilerplate code needed to fit and compare a number of models and allows the user to decide what to do with the results.

Poniard can be your first stab at a prediction problem, but it definitely shouldn't be your last one.

### Opinionated with a few exceptions
While some parameters can be modified to control how variable type inference and preprocessing are performed, the API is designed to prevent parameter proliferation.

### Cross validate all the things
Everything in Poniard is run with cross validation by default, and in fact no relevant functionality can be used without cross validation.

### Use baselines
A dummy estimator is always included in model comparisons so you can gauge whether your model is better than a dumb strategy.

### Fast TTFM (time to first model)
Preprocessing tries to ensure that your models run successfully without significant data munging. By default, Poniard imputes missing data and one-hot encodes or target encodes (depending on cardinality) inferred categorical variables, which in most cases is enough for scikit-learn algorithms to fit without complaints. Additionally, it scales numeric data and drops features with a single unique value.

## Similar projects
Poniard is not a groundbreaking idea, and a number of libraries follow a similar approach.

**[ATOM](https://github.com/tvdboom/ATOM)** is perhaps the most similar library to Poniard, albeit with a different approach to the API.

**[LazyPredict](https://github.com/shankarpandala/lazypredict)** is similar in that it runs multiple estimators and provides results for various metrics. Unlike Poniard, by default it tries most scikit-learn estimators, and is not based on cross validation.

**[PyCaret](https://github.com/pycaret/pycaret)** is a whole other beast that includes model explainability, deployment, plotting, NLP, anomaly detection, etc., which leads to a list of dependencies several times larger than Poniard's, and a more complicated API.